In [1]:
%matplotlib inline

import os
import sys
import math
import pandas as pd
import multiprocessing as mp
import matplotlib.pyplot as plt
from functools import partial

module_path = os.path.abspath(os.path.join('..'))
src_path = '{}/src'.format(module_path)
if module_path not in sys.path:
    sys.path.append(src_path)
    
from preprocessing import select
from consts import dataset, model_cols

plt.rcParams.update({'figure.max_open_warning': 0})

# Preparando a base de dados.
## Selecionando os atributos identificados como relevantes.

- Primeiro vamos definir as colunas a serem utilizadas;
- Depois, vamos utilizar
pequenos chunks para processar a base;
- Por fim, vamos salvar o arquivo
processado;
- No caso de já existir um arquivo CSV, ele será carregado.

In [2]:
save_path = '{}/data/processed/dataset.csv'.format(module_path)
load_path ='{}/data/raw/pessoas.txt'.format(module_path)

try:
    df_dataset = pd.read_csv(save_path)
except FileNotFoundError:
    # Preparando para processar o arquivo original.
    chunksize = 500
    pool = mp.Pool(mp.cpu_count())
    reader = pd.read_fwf(
        load_path,
        widths=dataset['col_widths'],
        names=dataset['col_names'],
        chunksize=chunksize
    )

    # Processamento do arquivo original.
    partial_select = partial(select, cols=model_cols)
    df_dataset = pd.concat(pool.map(partial_select, reader))
    df_dataset = df_dataset.replace('^\.$', '', regex=True)

    # Salvando arquivo processado.
    df_dataset.to_csv(save_path, header=True, index=False)

    # Dimensões da base
    df_dataset.shape

### Selecionando instâncias diagnosticadas com depressão

- Primeiro, será feita uma query buscando as instâncias onde o campo Diagnosticado_Depressao é igual 1;
- Depois, o resultado será salvo em um novo arquivo csv;
- Caso o arquivo já exista, basta ser carregado.

In [3]:
df_dep = df_dataset.query('Diagnosticado_Depressao == 1')
df_dep.shape

(4235, 68)

### Selecionando instâncias não diagnosticadas com depressão

- Primeiro, será feita uma query buscando as instâncias onde o campo Diagnosticado_Depressao é igual 2;
- Depois, o resultado será salvo em um novo arquivo csv;
- Caso o arquivo já exista, basta ser carregado.

In [4]:
df_ndep = df_dataset.query('Diagnosticado_Depressao == 2').sample(df_dep.shape[0])
df_ndep.shape

(4235, 68)

### Unindo as bases

O objetivo é trabalhar com instâncias diagnosticadas ou não com depressão. Para tanto, iremos concatenar as duas bases geradas.

In [5]:
sample_path = '{}/data/processed/sample.csv'.format(module_path)
df_sample = pd.concat([df_dep, df_ndep])
df_sample.to_csv(sample_path, header=True, index=False)
df_sample.shape

(8470, 68)

### Pré-processamento da base

(a) Os atributos "Aborto Espontâneo" e "Aborto Provocado" apresentam um valor alto de dados ausentes. Isso, porque boa parte dos campos não foram preenchidos com respostas de indivíduos do sexo masculino. Por isso, esses atributos devem levar em consideração o sexo e, portanto, devem ser transformados em algo como "Aborto Espontâneo/Sexo" e "Aborto Provocado/Sexo". Teremos, então, os valores 0 para masculino, 1 para feminino com aborto e 2 para feminino sem aborto

In [6]:
# Adicionando as coluna "Aborto_Espontaneo_Sexo"
df_sample = df_sample.assign(
    Aborto_Espontaneo_Sexo = lambda x: [ 0 if s == 1 else v for s, v in zip(x.Sexo, x.Aborto_Espontaneo) ]
)

# Adicionando as coluna "Aborto_Provocado_Sexo"
df_sample = df_sample.assign(
    Aborto_Provocado_Sexo = lambda x: [ 0 if s == 1 else v for s, v in zip(x.Sexo, x.Aborto_Provocado) ]
)

df_sample[['Aborto_Espontaneo_Sexo', 'Aborto_Provocado_Sexo']]

,Aborto_Espontaneo_Sexo,Aborto_Provocado_Sexo
1340,NaN,NaN
1358,0.0,0.0
1393,0.0,0.0
1547,1.0,2.0
1702,NaN,NaN
1967,1.0,2.0
2084,1.0,2.0
2326,2.0,2.0
2348,NaN,NaN
2424,1.0,2.0


In [7]:
n = df_sample.shape[0]
missing_vals = ((n - df_sample.count()) / df_sample.shape[0]) * 100
cols = { 0: 'Porcentagem' }

select(missing_vals.to_frame(), cols)

,Porcentagem
Regiao,0.000000
Diagnosticado_Depressao,0.000000
Defic_Intelectual,0.000000
Nasceu_Ou_Adquiriu_Defic_Intelectual,99.067296
Defic_Intelectual_Limita_Atividades,99.067296
Doença_Fisica_Cronica_Mental,0.000000
Visita_Medico,96.198347
Doença_Cerebral_Limita,96.198347
Esqz_Bipol_Psicose_TOC,0.000000
Psicoterapia_Doenca_Mental,96.198347


In [8]:
df_sample.loc[[1340, 1358]][
    ['Sexo', 'Aborto_Espontaneo', 'Aborto_Espontaneo_Sexo', 'Aborto_Provocado', 'Aborto_Provocado_Sexo']
]

,Sexo,Aborto_Espontaneo,Aborto_Espontaneo_Sexo,Aborto_Provocado,Aborto_Provocado_Sexo
1340,2,NaN,NaN,NaN,NaN
1358,1,NaN,0.0,NaN,0.0
